In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import matplotlib.pyplot as plt
import timeit
from PIL import Image

In [3]:
color = np.array(Image.open('../input/nyu-depth-v2/nyu_data/data/nyu2_test/00000_colors.png'))
depth = np.array(Image.open('../input/nyu-depth-v2/nyu_data/data/nyu2_test/00000_depth.png'))
color.shape,depth.shape

In [4]:
plt.figure()
plt.imshow(color)
plt.figure()
plt.imshow(depth)


In [5]:
min(depth.flatten()),max(depth.flatten())

In [6]:
depth

In [33]:
import os
import torch
import torch.nn as nn
import torchvision.models
import collections
import math
import torch.nn.functional as F
import torchvision.transforms as transforms
import time

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [9]:
def weights_init(m):
    # Initialize kernel weights with Gaussian distributions
    if isinstance(m, nn.Conv2d):
        n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        m.weight.data.normal_(0, math.sqrt(2. / n))
        if m.bias is not None:
            m.bias.data.zero_()
    elif isinstance(m, nn.ConvTranspose2d):
        n = m.kernel_size[0] * m.kernel_size[1] * m.in_channels
        m.weight.data.normal_(0, math.sqrt(2. / n))
        if m.bias is not None:
            m.bias.data.zero_()
    elif isinstance(m, nn.BatchNorm2d):
        m.weight.data.fill_(1)
        m.bias.data.zero_()
        
def conv(in_channels, out_channels, kernel_size):
    padding = (kernel_size-1) // 2
    assert 2*padding == kernel_size-1, "parameters incorrect. kernel={}, padding={}".format(kernel_size, padding)
    return nn.Sequential(
          nn.Conv2d(in_channels,out_channels,kernel_size,stride=1,padding=padding,bias=False),
          nn.BatchNorm2d(out_channels),
          nn.ReLU(inplace=True),
        )

def depthwise(in_channels, kernel_size):
    padding = (kernel_size-1) // 2
    assert 2*padding == kernel_size-1, "parameters incorrect. kernel={}, padding={}".format(kernel_size, padding)
    return nn.Sequential(
          nn.Conv2d(in_channels,in_channels,kernel_size,stride=1,padding=padding,bias=False,groups=in_channels),
          nn.BatchNorm2d(in_channels),
          nn.ReLU(inplace=True),
        )

def pointwise(in_channels, out_channels):
    return nn.Sequential(
          nn.Conv2d(in_channels,out_channels,1,1,0,bias=False),
          nn.BatchNorm2d(out_channels),
          nn.ReLU(inplace=True),
        )



In [10]:
class MobileNet(nn.Module):
    def __init__(self, relu6=True):
        super(MobileNet, self).__init__()

        def relu(relu6):
            if relu6:
                return nn.ReLU6(inplace=True)
            else:
                return nn.ReLU(inplace=True)

        def conv_bn(inp, oup, stride, relu6):
            return nn.Sequential(
                nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
                nn.BatchNorm2d(oup),
                relu(relu6),
            )

        def conv_dw(inp, oup, stride, relu6):
            return nn.Sequential(
                nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
                nn.BatchNorm2d(inp),
                relu(relu6),
    
                nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
                relu(relu6),
            )

        self.model = nn.Sequential(
            conv_bn(  3,  32, 2, relu6), 
            conv_dw( 32,  64, 1, relu6),
            conv_dw( 64, 128, 2, relu6),
            conv_dw(128, 128, 1, relu6),
            conv_dw(128, 256, 2, relu6),
            conv_dw(256, 256, 1, relu6),
            conv_dw(256, 512, 2, relu6),
            conv_dw(512, 512, 1, relu6),
            conv_dw(512, 512, 1, relu6),
            conv_dw(512, 512, 1, relu6),
            conv_dw(512, 512, 1, relu6),
            conv_dw(512, 512, 1, relu6),
            conv_dw(512, 1024, 2, relu6),
            conv_dw(1024, 1024, 1, relu6),
            nn.AvgPool2d(7),
        )
        self.fc = nn.Linear(1024, 1000)

    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, 1024)
        x = self.fc(x)
        return x

print('num parameters:', sum(p.numel() for p in MobileNet().parameters() if p.requires_grad))
print('\n',MobileNet())

In [11]:
class Depth_Estimator(nn.Module):
    def __init__(self):
        super(Depth_Estimator, self).__init__()
        mobilenet = MobileNet()
        # pretrained
        pretrained_path = '../input/mobilenet-pretrained/mobilenet_model.pth.tar'
        checkpoint = torch.load(pretrained_path)
        state_dict = checkpoint['state_dict']

        from collections import OrderedDict
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            name = k[7:] # remove `module.`
            new_state_dict[name] = v
        mobilenet.load_state_dict(new_state_dict)


        for i in range(14):
            setattr( self, 'conv{}'.format(i), mobilenet.model[i])

        # NNconv5
        kernel_size = 5
        self.decode_conv1 = nn.Sequential(
            depthwise(1024, kernel_size),
            pointwise(1024, 512))
        self.decode_conv2 = nn.Sequential(
            depthwise(512, kernel_size),
            pointwise(512, 256))
        self.decode_conv3 = nn.Sequential(
            depthwise(256, kernel_size),
            pointwise(256, 128))
        self.decode_conv4 = nn.Sequential(
            depthwise(128, kernel_size),
            pointwise(128, 64))
        self.decode_conv5 = nn.Sequential(
            depthwise(64, kernel_size),
            pointwise(64, 32))
        self.decode_conv6 = pointwise(32, 1)
        weights_init(self.decode_conv1)
        weights_init(self.decode_conv2)
        weights_init(self.decode_conv3)
        weights_init(self.decode_conv4)
        weights_init(self.decode_conv5)
        weights_init(self.decode_conv6)

    def forward(self, x):
        # encoder and preparing skip connections
        for i in range(14):
            layer = getattr(self, 'conv{}'.format(i))
            x = layer(x)
            # print("{}: {}".format(i, x.size()))
            if i==1:
                x1 = x
            elif i==3:
                x2 = x
            elif i==5:
                x3 = x
                
        # decoder with NNconv5 and additive skip connections
        for i in range(1,6):
            layer = getattr(self, 'decode_conv{}'.format(i))
            x = layer(x)
            x = F.interpolate(x, scale_factor=2, mode='nearest')
            if i==4:
                x = x + x1
            elif i==3:
                x = x + x2
            elif i==2:
                x = x + x3
            # print("{}: {}".format(i, x.size()))
        x = self.decode_conv6(x)
        return x
    
Model_Estimator = Depth_Estimator()
print('\n','num parameters:', sum(p.numel() for p in Model_Estimator.parameters() if p.requires_grad))
print(Model_Estimator)


In [27]:
import pandas as pd

df_tr = pd.read_csv('../input/nyu-depth-v2/nyu_data/data/nyu2_train.csv', header=None)
df_tr.rename(columns = {0:'data', 1:'target'}, inplace = True)
df_test = pd.read_csv('../input/nyu-depth-v2/nyu_data/data/nyu2_test.csv', header=None)
df_test.rename(columns = {0:'data', 1:'target'}, inplace = True)
df_test = df_test.reset_index(drop=True)

len_tr = len(df_tr)
len_test = len(df_test)

df_tr['data'][0],df_tr['target'][0]

In [28]:
train_list = np.random.choice(len_tr,len_tr//30, replace=False)
len_ = len(train_list)
ind_shuffle = np.arange(len_)
np.random.shuffle(ind_shuffle)
train_list = train_list[ind_shuffle]

df_train =df_tr.iloc[train_list[:len_//5]]

df_val =df_tr.iloc[train_list[len_//5:]]

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

len_train = len(df_train)
len_val = len(df_val)
len_train,len_val,len_test

In [ ]:
class preparing_data(torch.utils.data.Dataset):
    def __init__(self, subset: str ,root = '../input/nyu-depth-v2/nyu_data' ):

        self.root = root
        self.transform = transforms.Compose([
            transforms.CenterCrop((336, 448)),
            transforms.Resize(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])
        if subset == "train":
            self.datasets = df_train
                
        elif subset == "validation":
            self.datasets = df_val
        
        elif  subset == "test":
            self.datasets = df_test
    
                
        else :
            raise ValueError(f"Unknown subset {subset}. Use validation/testing/training")
            
    def train_transform(self, data_, label_):
        # perform 1st step of data augmentation
        return  self.transform(label_)

    def __len__(self):
        return len(self.datasets)
    
    def __getitem__(self,n):
        
        data = Image.open(os.path.join(self.root, self.datasets['data'][n])) 
        label = Image.open(os.path.join(self.root, self.datasets['target'][n])) 
        data, label  = self.train_transform(data, label)

        return data, label

In [ ]:
a = preparing_data('train')

In [ ]:
a[0]